# SIMD

SIMD stands for **"Single Instruction Multiple Data"** and falls into the category of instruction level parallelism (vector instructions). Consider this simple example where `A`, `B`, and `C` are vectors:

In [1]:
function vector_add(A, B, C)
    for i in eachindex(A, B, C)
        @inbounds A[i] = B[i] + C[i]
    end
end

vector_add (generic function with 1 method)


The idea behind SIMD is to perform the add instruction on multiple elements at the same time (instead of separately performing them one after another). The process of splitting up the simple loop addition into multiple vector additions is often denoted as "loop vectorization". Since each vectorized addition happens at instruction level, i.e. within a CPU core, the feature set of the CPU determines how many elements we can process in one go.

<img src="../../imgs/simd_vaddpd.png" width=300px>
<img src="../../imgs/simd_register_width.png" width=400px>

(**Source:** Node-level performance engineering course by [NHR@FAU](https://hpc.fau.de/))

Let's check which "advanced vector extensions" (AVX) the system supports.

In [64]:
using CpuId
cpuinfo()

| Cpu Property       | Value                                                      |
|:------------------ |:---------------------------------------------------------- |
| Brand              | Intel(R) Xeon(R) Gold 6148F CPU @ 2.40GHz                  |
| Vendor             | :Intel                                                     |
| Architecture       | :Skylake                                                   |
| Model              | Family: 0x06, Model: 0x55, Stepping: 0x04, Type: 0x00      |
| Cores              | 20 physical cores, 40 logical cores (on executing CPU)     |
|                    | Hyperthreading hardware capability detected                |
| Clock Frequencies  | 2400 / 3700 MHz (base/max), 100 MHz bus                    |
| Data Cache         | Level 1:3 : (32, 1024, 28160) kbytes                       |
|                    | 64 byte cache line size                                    |
| Address Size       | 48 bits virtual, 46 bits physical                          |
| SIMD               | 512 bit = 64 byte max. SIMD vector size                    |
| Time Stamp Counter | TSC is accessible via `rdtsc`                              |
|                    | TSC runs at constant rate (invariant from clock frequency) |
| Perf. Monitoring   | Performance Monitoring Counters (PMC) revision 4           |
|                    | Available hardware counters per logical core:              |
|                    | 3 fixed-function counters of 48 bit width                  |
|                    | 8 general-purpose counters of 48 bit width                 |
| Hypervisor         | No                                                         |


In [65]:
filter(x -> contains(string(x), "AVX"), cpufeatures())

7-element Vector{Symbol}:
 :AVX
 :AVX2
 :AVX512BW
 :AVX512CD
 :AVX512DQ
 :AVX512F
 :AVX512VL

In [66]:
SIZE = 512 * 100

51200

In [67]:
A = rand(Float64, SIZE)
B = rand(Float64, SIZE)
C = rand(Float64, SIZE);

In [68]:
@code_native debuginfo=:none syntax=:intel vector_add(A,B,C)

	.text
	.file	"vector_add"
	.globl	japi1_vector_add_6155           # -- Begin function japi1_vector_add_6155
	.p2align	4, 0x90
	.type	japi1_vector_add_6155,@function
japi1_vector_add_6155:                  # @japi1_vector_add_6155
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 160
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp + 32], ymm0
	mov	qword ptr [rsp + 64], 0
	mov	qword ptr [rsp + 128], rsi
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r12, qword ptr [rax - 8]
	mov	qword ptr [rsp + 32], 12
	mov	rax, qword ptr [r12]
	mov	qword ptr [rsp + 40], rax
	lea	rax, [rsp + 32]
	mov	qword ptr [r12], rax
	mov	rdx, qword ptr [rsi]
	mov	rax, qword ptr [rsi + 8]
	mov	rcx, qword ptr [rsi + 16]
	mov

## It's not always so simple: Reduction

In [69]:
function vector_dot(B, C)
    a = zero(eltype(B))
    for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot (generic function with 1 method)

In [70]:
@code_native debuginfo=:none syntax=:intel vector_dot(B, C)

	.text
	.file	"vector_dot"
	.globl	julia_vector_dot_6178           # -- Begin function julia_vector_dot_6178
	.p2align	4, 0x90
	.type	julia_vector_dot_6178,@function
julia_vector_dot_6178:                  # @julia_vector_dot_6178
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r15], rax
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_6
# %bb.1:                                # %L26
	test	rbx, rbx
	je	.LBB0_2
# %bb.3:   

Note the `vaddsd` instruction and usage of `xmmi` registers (128 bit).

How could this loop be vectorized?

In [71]:
function vector_dot_unrolled4(B, C)
    a1 = zero(eltype(B))
    a2 = zero(eltype(B))
    a3 = zero(eltype(B))
    a4 = zero(eltype(B))
    @inbounds for i in 1:4:length(B)-4
        a1 += B[i] * C[i]
        a2 += B[i+1] * C[i+1]
        a3 += B[i+2] * C[i+2]
        a4 += B[i+3] * C[i+3]
    end
    return a1+a2+a3+a4
end

vector_dot_unrolled4 (generic function with 1 method)

In [72]:
@code_native debuginfo=:none syntax=:intel vector_dot_unrolled4(B, C)

	.text
	.file	"vector_dot_unrolled4"
	.globl	julia_vector_dot_unrolled4_6202 # -- Begin function julia_vector_dot_unrolled4_6202
	.p2align	4, 0x90
	.type	julia_vector_dot_unrolled4_6202,@function
julia_vector_dot_unrolled4_6202:        # @julia_vector_dot_unrolled4_6202
	.cfi_startproc
# %bb.0:                                # %top
	push	r14
	.cfi_def_cfa_offset 16
	push	rbx
	.cfi_def_cfa_offset 24
	sub	rsp, 8
	.cfi_def_cfa_offset 32
	.cfi_offset rbx, -24
	.cfi_offset r14, -16
	mov	r14, rsi
	mov	rbx, rdi
	mov	rdx, qword ptr [rdi + 8]
	add	rdx, -4
	movabs	rax, offset j_steprange_last_6204
	mov	edi, 1
	mov	esi, 4
	call	rax
	test	rax, rax
	jle	.LBB0_1
# %bb.2:                                # %L19.preheader
	mov	rcx, qword ptr [rbx]
	mov	rdx, qword ptr [r14]
	add	rax, 3
	vxorpd	xmm0, xmm0, xmm0
	xor	esi, esi
	.p2align	4, 0x90
.LBB0_3:                                # %L19
                                        # =>This Inner Loop Header: Depth=1
	vmovupd	ymm1, ymmword ptr [rcx + 8*rsi]
	

In [73]:
using BenchmarkTools
@btime vector_dot($B, $C);
@btime vector_dot_unrolled4($B, $C);

  55.521 μs (0 allocations: 0 bytes)
  14.395 μs (0 allocations: 0 bytes)


To "force" automatic SIMD vectorization in Julia, you can use the `@simd` macro.

In [74]:
function vector_dot_simd(B, C)
    a = zero(eltype(B))
    @simd for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot_simd (generic function with 1 method)

By using the `@simd` macro, we are asserting several properties of the loop:

* It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
* Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In [75]:
@btime vector_dot_simd($B, $C);

  9.293 μs (0 allocations: 0 bytes)


This is a huge speedup for just a little extra `@simd`!

In [76]:
@code_native debuginfo=:none syntax=:intel vector_dot_simd(B, C)

	.text
	.file	"vector_dot_simd"
	.globl	julia_vector_dot_simd_6281      # -- Begin function julia_vector_dot_simd_6281
	.p2align	4, 0x90
	.type	julia_vector_dot_simd_6281,@function
julia_vector_dot_simd_6281:             # @julia_vector_dot_simd_6281
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r15], rax
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_10
# %bb.1:                                # %L28
	test	rbx, rbx
	j

Note the `vfmadd231pd` instruction and usage of `ymmi` AVX registers (256 bit).

Data types matter:
* Floating-point addition is **non-associative** and the order of operations is important.
* Integer addition is **associative** and the order of operations has no impact.

Let's check what happens for `Int64` input.

In [77]:
B_int = rand(Int64, SIZE)
C_int = rand(Int64, SIZE)
@btime vector_dot($B_int, $C_int);
@btime vector_dot_simd($B_int, $C_int);

  11.050 μs (0 allocations: 0 bytes)
  11.078 μs (0 allocations: 0 bytes)


As expected, there is no difference between the two variants.

### SIMD is hard...

* Autovectorization is a hard problem (it needs to prove a lot of things about the code!)
* Not every code / loop is readily vectorizable
  * Keep your loops simple, e.g. avoid conditionals, control flow, and function calls if possible!
  * Loop length should be countable up front
  * Contiguous data access
  * (Align data structures to SIMD width boundary)

**Keep it simple!**

### [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)

Think of `@turbo` as a more sophisticated version of `@simd`. Hopefully, these features will at some point just be part of Julia's compiler.

In [79]:
using LoopVectorization

function vector_dot_turbo(B, C)
    a = zero(eltype(B))
    @turbo for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

@btime vector_dot_simd($B, $C);
@btime vector_dot_turbo($B, $C);

  9.297 μs (0 allocations: 0 bytes)
  6.331 μs (0 allocations: 0 bytes)


In [80]:
@code_native debuginfo=:none syntax=:intel vector_dot_turbo(B, C)

	.text
	.file	"vector_dot_turbo"
	.globl	julia_vector_dot_turbo_6512     # -- Begin function julia_vector_dot_turbo_6512
	.p2align	4, 0x90
	.type	julia_vector_dot_turbo_6512,@function
julia_vector_dot_turbo_6512:            # @julia_vector_dot_turbo_6512
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r15], rax
	mov	r12, qword ptr [rdi + 8]
	mov	rbx, qword ptr [rsi + 8]
	cmp	r12, rbx
	jne	.LBB0_14
# %bb.1:                                # %L32
	movabs	rbx, 

Note the usage of the `zmmi` AVX512 registers! (512 bit)

## Structure of Array vs Array of Structure

Data layout can matter!

In [101]:
# Array of structure
AoS = [complex(rand(),rand()) for i in 1:SIZE]

51200-element Vector{ComplexF64}:
   0.6118049598601762 + 0.8536982158092113im
  0.06975145408896988 + 0.6378074534656512im
  0.19980319856163942 + 0.713440640999059im
  0.10101616584846729 + 0.21614202906466506im
   0.7836387038522062 + 0.005947223529423229im
    0.388050530478658 + 0.10130749016030716im
   0.8089462249702304 + 0.003234986558072195im
  0.06950366802448227 + 0.5308309613486356im
  0.36566424785573026 + 0.18185495157041853im
  0.10350044554804927 + 0.3256597424080613im
                      ⋮
   0.8828535299141503 + 0.7232214761323134im
   0.7340014363483335 + 0.16868353705606975im
   0.8485240348220299 + 0.927671845557969im
  0.14894152489435653 + 0.13279162350786522im
   0.7909127665639123 + 0.6267623351645433im
 0.044503651354593665 + 0.91240304867587im
    0.255426299323801 + 0.183458989629396im
   0.8523984559786467 + 0.11417197997728701im
   0.6508067809207818 + 0.3749041127417122im

In [102]:
@btime sum($AoS);

  15.833 μs (0 allocations: 0 bytes)


### [StructArrays.jl](https://github.com/JuliaArrays/StructArrays.jl)

In [83]:
using StructArrays

In [103]:
SoA = StructArray{Complex}((rand(SIZE), rand(SIZE)))

51200-element StructArray(::Vector{Float64}, ::Vector{Float64}) with eltype Complex:
  0.6752932843739214 + 0.3740534765644722im
  0.9133277434458186 + 0.04237138799660711im
 0.42766703314137977 + 0.5806069884469464im
 0.23846156503652116 + 0.9055620820351987im
  0.8758282747071988 + 0.38373308355279556im
  0.5102917564236148 + 0.9902329671462641im
  0.9406760055730625 + 0.06391341806269157im
  0.4605806811378489 + 0.1631384926741708im
  0.6513022153930962 + 0.2988130428834155im
   0.726751046419119 + 0.5574045066340347im
                     ⋮
 0.23390987977869948 + 0.6836293205101093im
  0.5262189853387025 + 0.20462208709362195im
 0.35371784657163474 + 0.8225701898433732im
  0.6612915646192037 + 0.32757967013126477im
 0.48720720888974056 + 0.8955974530405787im
  0.9550500600454312 + 0.15531406346372012im
  0.8223482288318515 + 0.06016168340051198im
  0.9805717769539437 + 0.1291311284286223im
  0.2592282899440891 + 0.6076283714202433im

In [104]:
@btime sum($SoA);

  11.092 μs (0 allocations: 0 bytes)


**Resources:**

* [LoopVectorization.jl video on youtube](https://www.youtube.com/watch?v=qz2kJdVDWi0)
* [SIMD and SIMD-intrinsics in Julia](http://kristofferc.github.io/post/intrinsics/)
* [Optimizing Serial Code](https://mitmath.github.io/18337/lecture2/optimizing)